In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-eu.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [ ]:
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 217.8MB 57kB/s 
     |████████████████████████████████| 204kB 51.6MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=fbe9cefbe1a4ebb4e28701014d5eb6f73e4e9735dca0c83c77e4898351ffcfcf
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
APP_NAME = "DataFramesAndMLLib"
SPARK_URL = "local[*]"

In [ ]:
spark = SparkSession.builder.appName(APP_NAME).getOrCreate()

In [ ]:
import numpy as np
import pandas as pd
df = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))

spark_df = spark.createDataFrame(df)

In [ ]:
spark_df.show(10)

+---+---+---+---+
|  A|  B|  C|  D|
+---+---+---+---+
| 55| 53| 92| 67|
| 38|  7| 77| 67|
| 91| 52| 76| 47|
| 14| 23| 64|  8|
| 69| 25| 51| 16|
| 92| 43| 73| 75|
| 77| 98| 93| 55|
| 40| 64| 96| 86|
|  5| 54| 19| 44|
| 37| 14| 38| 78|
+---+---+---+---+
only showing top 10 rows



In [ ]:
spark_df.take(3)

[Row(A=55, B=53, C=92, D=67),
 Row(A=38, B=7, C=77, D=67),
 Row(A=91, B=52, C=76, D=47)]

In [ ]:
location = './gdrive/My Drive/Colab Datasets/Video_Games_Sales_as_at_22_Dec_2016.csv'
video = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load(location)

In [ ]:
video.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year_of_Release: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: double (nullable = true)
 |-- EU_Sales: double (nullable = true)
 |-- JP_Sales: double (nullable = true)
 |-- Other_Sales: double (nullable = true)
 |-- Global_Sales: double (nullable = true)
 |-- Critic_Score: integer (nullable = true)
 |-- Critic_Count: integer (nullable = true)
 |-- User_Score: string (nullable = true)
 |-- User_Count: integer (nullable = true)
 |-- Developer: string (nullable = true)
 |-- Rating: string (nullable = true)



In [ ]:
video.show()

+--------------------+--------+---------------+------------+--------------------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+-------------------+------+
|                Name|Platform|Year_of_Release|       Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|Critic_Score|Critic_Count|User_Score|User_Count|          Developer|Rating|
+--------------------+--------+---------------+------------+--------------------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+-------------------+------+
|          Wii Sports|     Wii|           2006|      Sports|            Nintendo|   41.36|   28.96|    3.77|       8.45|       82.53|          76|          51|         8|       322|           Nintendo|     E|
|   Super Mario Bros.|     NES|           1985|    Platform|            Nintendo|   29.08|    3.58|    6.81|       0.77|       40.24|        null|        null|     

In [ ]:
video_renamed = video.withColumnRenamed("NA_Sales", "North_America_Sales")

In [ ]:
video_renamed.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year_of_Release: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- North_America_Sales: double (nullable = true)
 |-- EU_Sales: double (nullable = true)
 |-- JP_Sales: double (nullable = true)
 |-- Other_Sales: double (nullable = true)
 |-- Global_Sales: double (nullable = true)
 |-- Critic_Score: integer (nullable = true)
 |-- Critic_Count: integer (nullable = true)
 |-- User_Score: string (nullable = true)
 |-- User_Count: integer (nullable = true)
 |-- Developer: string (nullable = true)
 |-- Rating: string (nullable = true)



In [ ]:
from pyspark.sql.types import StructType, StructField, FloatType, StringType

In [ ]:
irisSchema = StructType([
  StructField('petal_length', FloatType(), True),
  StructField('petal_width', FloatType(), True),
  StructField('sepal_length', FloatType(), True),
  StructField('sepal_width', FloatType(), True),
  StructField('variety', StringType(), True)
])

In [ ]:
iris = spark.read.format("csv").option("header", "true").schema(irisSchema).load("./gdrive/My Drive/Colab Datasets/iris.csv")

iris.show(5)

+------------+-----------+------------+-----------+-------+
|petal_length|petal_width|sepal_length|sepal_width|variety|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| Setosa|
|         4.9|        3.0|         1.4|        0.2| Setosa|
|         4.7|        3.2|         1.3|        0.2| Setosa|
|         4.6|        3.1|         1.5|        0.2| Setosa|
|         5.0|        3.6|         1.4|        0.2| Setosa|
+------------+-----------+------------+-----------+-------+
only showing top 5 rows



In [ ]:
video_other_ratio = video.withColumn("Other_Sales_Ratio", video['Other_Sales'] / video['Global_Sales'])

In [ ]:
video_other_ratio.select("Other_Sales_Ratio").show(5)

+--------------------+
|   Other_Sales_Ratio|
+--------------------+
| 0.10238701078395734|
|0.019135188866799203|
| 0.09262387387387387|
| 0.09002136100091547|
|0.031877590054191905|
+--------------------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import lit

In [ ]:
video_lit = video_other_ratio.withColumn("Other_Sales_Percent", video_other_ratio['Other_Sales_Ratio'] * lit(100))

In [ ]:
video_lit.select("Other_Sales_Percent").show(5)

+-------------------+
|Other_Sales_Percent|
+-------------------+
| 10.238701078395735|
| 1.9135188866799202|
|  9.262387387387387|
|  9.002136100091548|
| 3.1877590054191907|
+-------------------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import IntegerType

In [ ]:
nintendo = UserDefinedFunction(lambda x: 1 if x == "Nintendo" else 0)

In [ ]:
video_nintendo = video.withColumn("Nintendo", nintendo(video['Publisher']))

In [ ]:
video_nintendo.show(5)

+--------------------+--------+---------------+------------+---------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+---------+------+--------+
|                Name|Platform|Year_of_Release|       Genre|Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|Critic_Score|Critic_Count|User_Score|User_Count|Developer|Rating|Nintendo|
+--------------------+--------+---------------+------------+---------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+---------+------+--------+
|          Wii Sports|     Wii|           2006|      Sports| Nintendo|   41.36|   28.96|    3.77|       8.45|       82.53|          76|          51|         8|       322| Nintendo|     E|       1|
|   Super Mario Bros.|     NES|           1985|    Platform| Nintendo|   29.08|    3.58|    6.81|       0.77|       40.24|        null|        null|      null|      null|     null|  null|       1|
|      Mario Ka

In [ ]:
video.registerTempTable("video")

In [ ]:
spark.sql("SELECT * FROM video WHERE NA_Sales > 25").show()

+-----------------+--------+---------------+--------+---------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+---------+------+
|             Name|Platform|Year_of_Release|   Genre|Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|Critic_Score|Critic_Count|User_Score|User_Count|Developer|Rating|
+-----------------+--------+---------------+--------+---------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+---------+------+
|       Wii Sports|     Wii|           2006|  Sports| Nintendo|   41.36|   28.96|    3.77|       8.45|       82.53|          76|          51|         8|       322| Nintendo|     E|
|Super Mario Bros.|     NES|           1985|Platform| Nintendo|   29.08|    3.58|    6.81|       0.77|       40.24|        null|        null|      null|      null|     null|  null|
|        Duck Hunt|     NES|           1984| Shooter| Nintendo|   26.93|    0.63|    0.28|     

In [ ]:
video.groupBy("Publisher").sum("Global_Sales").orderBy('sum(Global_Sales)', ascending=False).show(5)

+--------------------+------------------+
|           Publisher| sum(Global_Sales)|
+--------------------+------------------+
|            Nintendo|1788.8099999999977|
|     Electronic Arts|1116.9599999999898|
|          Activision| 731.1599999999987|
|Sony Computer Ent...| 606.4799999999988|
|             Ubisoft| 471.6099999999993|
+--------------------+------------------+
only showing top 5 rows



In [ ]:
video.select("Genre").distinct().show()

+------------+
|       Genre|
+------------+
|   Adventure|
|      Sports|
|        null|
|      Racing|
|Role-Playing|
|     Shooter|
|        Misc|
|    Platform|
|      Puzzle|
|    Fighting|
|      Action|
|    Strategy|
|  Simulation|
+------------+



In [ ]:
video.select("Genre").distinct().count()

13

In [ ]:
video.count()

16719

In [ ]:
video.dropna().count()

6947

In [ ]:
video.dropna(thresh=12).count()

10075

In [ ]:
video.fillna(0)

DataFrame[Name: string, Platform: string, Year_of_Release: string, Genre: string, Publisher: string, NA_Sales: double, EU_Sales: double, JP_Sales: double, Other_Sales: double, Global_Sales: double, Critic_Score: int, Critic_Count: int, User_Score: string, User_Count: int, Developer: string, Rating: string]

In [ ]:
video.groupBy("Publisher").sum("Global_Sales").orderBy('sum(Global_Sales)', ascending=False).explain()

== Physical Plan ==
*(3) Sort [sum(Global_Sales)#226 DESC NULLS LAST], true, 0
+- Exchange rangepartitioning(sum(Global_Sales)#226 DESC NULLS LAST, 200)
   +- *(2) HashAggregate(keys=[Publisher#14], functions=[sum(Global_Sales#19)])
      +- Exchange hashpartitioning(Publisher#14, 200)
         +- *(1) HashAggregate(keys=[Publisher#14], functions=[partial_sum(Global_Sales#19)])
            +- *(1) FileScan csv [Publisher#14,Global_Sales#19] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/content/gdrive/My Drive/Colab Datasets/Video_Games_Sales_as_at_22_Dec_201..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Publisher:string,Global_Sales:double>


In [ ]:
location = './gdrive/My Drive/Colab Datasets/iris.csv'
iris = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load(location)

In [ ]:
iris.show(5)

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|variety|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| Setosa|
|         4.9|        3.0|         1.4|        0.2| Setosa|
|         4.7|        3.2|         1.3|        0.2| Setosa|
|         4.6|        3.1|         1.5|        0.2| Setosa|
|         5.0|        3.6|         1.4|        0.2| Setosa|
+------------+-----------+------------+-----------+-------+
only showing top 5 rows



In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
feature_cols = [x for x in iris.columns if x != 'variety']
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')

iris_features = assembler.transform(iris)

In [ ]:
iris_features.show(5)

+------------+-----------+------------+-----------+-------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|variety|         features|
+------------+-----------+------------+-----------+-------+-----------------+
|         5.1|        3.5|         1.4|        0.2| Setosa|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2| Setosa|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2| Setosa|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2| Setosa|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2| Setosa|[5.0,3.6,1.4,0.2]|
+------------+-----------+------------+-----------+-------+-----------------+
only showing top 5 rows



In [ ]:
from pyspark.ml.clustering import KMeans

In [ ]:
kmeans = KMeans().setK(3).setSeed(1)
model = kmeans.fit(iris_features.select("features"))

In [ ]:
centers = model.clusterCenters()

In [ ]:
print(centers)

[array([5.88360656, 2.74098361, 4.38852459, 1.43442623]), array([6.85384615, 3.07692308, 5.71538462, 2.05384615]), array([5.006, 3.428, 1.462, 0.246])]


In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
indexer = StringIndexer(inputCol='variety', outputCol='label')
index_model = indexer.fit(iris_features)
iris_input = index_model.transform(iris_features).select("features", "label")

In [ ]:
iris_input.show(5)

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[5.1,3.5,1.4,0.2]|  2.0|
|[4.9,3.0,1.4,0.2]|  2.0|
|[4.7,3.2,1.3,0.2]|  2.0|
|[4.6,3.1,1.5,0.2]|  2.0|
|[5.0,3.6,1.4,0.2]|  2.0|
+-----------------+-----+
only showing top 5 rows



In [ ]:
train, test = iris_input.randomSplit([0.7, 0.3], seed = 1)

print(train.count(), test.count())

106 44


In [ ]:
from pyspark.ml.classification import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(featuresCol='features', labelCol='label')
rfModel = rf.fit(train)

In [ ]:
prediction = rfModel.transform(test)

In [ ]:
prediction_rdd = prediction.select("prediction", "label").rdd.map(tuple)
prediction_rdd.take(2)

[(2.0, 2.0), (2.0, 2.0)]

In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics

metrics = MulticlassMetrics(prediction_rdd)
metrics.accuracy

0.9318181818181818

In [ ]:
insurance_location = './gdrive/My Drive/Drexel Big Data/Data/insurance.csv'
insurance = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load(insurance_location)

In [ ]:
insurance.show(5)

+---+------+------+--------+------+---------+-----------+
|age|   sex|   bmi|children|smoker|   region|    charges|
+---+------+------+--------+------+---------+-----------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|
| 33|  male|22.705|       0|    no|northwest|21984.47061|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|
+---+------+------+--------+------+---------+-----------+
only showing top 5 rows



In [ ]:
from pyspark.ml.feature import StringIndexer

string_index = StringIndexer(inputCol='region', outputCol='region_index').fit(insurance)

string_index.transform(insurance).show(5)

+---+------+------+--------+------+---------+-----------+------------+
|age|   sex|   bmi|children|smoker|   region|    charges|region_index|
+---+------+------+--------+------+---------+-----------+------------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|         1.0|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|         0.0|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|         0.0|
| 33|  male|22.705|       0|    no|northwest|21984.47061|         2.0|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|         2.0|
+---+------+------+--------+------+---------+-----------+------------+
only showing top 5 rows



In [ ]:
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

cols = ["smoker", "sex", "region"]
stages = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(insurance) for column in cols]
stages.append(OneHotEncoderEstimator(inputCols=["region_index"], outputCols=["region_num"]))

feature_cols = ['age', 'bmi', 'children', 'smoker_index', 'sex_index', 'region_num']
stages.append(VectorAssembler(inputCols=feature_cols, outputCol='features'))

pipeline = Pipeline(stages=stages)
insurance_transformed = pipeline.fit(insurance).transform(insurance)

insurance_transformed.show(5)


+---+------+------+--------+------+---------+-----------+------------+---------+------------+-------------+--------------------+
|age|   sex|   bmi|children|smoker|   region|    charges|smoker_index|sex_index|region_index|   region_num|            features|
+---+------+------+--------+------+---------+-----------+------------+---------+------------+-------------+--------------------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|         1.0|      1.0|         1.0|(3,[1],[1.0])|[19.0,27.9,0.0,1....|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|         0.0|      0.0|         0.0|(3,[0],[1.0])|(8,[0,1,2,5],[18....|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|         0.0|      0.0|         0.0|(3,[0],[1.0])|(8,[0,1,2,5],[28....|
| 33|  male|22.705|       0|    no|northwest|21984.47061|         0.0|      0.0|         2.0|(3,[2],[1.0])|(8,[0,1,7],[33.0,...|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|         0.0|      0.0|         2.0|(3,[

In [ ]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol = 'features', labelCol='charges', maxIter=10)
lr_model = lr.fit(insurance_transformed)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [256.85635253734966,339.19345361083236,475.50054514913035,23848.534541912828,131.31435939505056,-1035.022049387815,-960.0509913008347,-352.96389942466766]
Intercept: -12069.85293556212
